In [ ]:
import os
import json

In [ ]:
# TRAIN_IMAGE_FOLDER = "./train/images/"
# TRAIN_LABEL_FOLDER = "./train/annotations/"

# TEST_IMAGE_FOLDER = "./test/images/"
# TEST_LABEL_FOLDER = "./test/annotations/"

# ROOT_DATA_FOLDER = "./external_data/all/"
# ROOT_DATA_FOLDER = "./external_data/db/"
ROOT_DATA_FOLDER = "./"

TRAIN_IMAGE_FOLDER = os.path.join(ROOT_DATA_FOLDER, "train/images/")
TRAIN_LABEL_FOLDER = os.path.join(ROOT_DATA_FOLDER, "train/annotations/")

TEST_IMAGE_FOLDER = os.path.join(ROOT_DATA_FOLDER, "test/images/")
TEST_LABEL_FOLDER = os.path.join(ROOT_DATA_FOLDER, "test/annotations/")

In [ ]:
image_paths = []
label_paths = []

for file in os.listdir(TRAIN_IMAGE_FOLDER):
    image_paths.append(os.path.join(TRAIN_IMAGE_FOLDER, file))

for file in os.listdir(TRAIN_LABEL_FOLDER):
    label_paths.append(os.path.join(TRAIN_LABEL_FOLDER, file))

print("Number of images: ", len(image_paths))
print("Number of labels: ", len(label_paths))

In [ ]:
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure


ground_truths = []
keypoints_ground_truths = {}
text_detection_ground_truths = {}
x_labels_text_detection_ground_truths = {}
y_labels_text_detection_ground_truths = {}

for path in tqdm(label_paths):
    # if "17000b60f53e" not in path:
    #     continue
    with open(path) as json_file:
        data = json.load(json_file)

    if os.path.exists(os.path.join(TRAIN_IMAGE_FOLDER, os.path.basename(path)[:-5] + ".jpg")):
        image_name = os.path.basename(path)[:-5] + ".jpg"
    elif os.path.exists(os.path.join(TRAIN_IMAGE_FOLDER, os.path.basename(path)[:-5] + ".png")):
        image_name = os.path.basename(path)[:-5] + ".png"
    else:
        print("Image not found: ", path)
        continue

    for item in data["data-series"]:
        for k in item:
            if isinstance(item[k], float):
                item[k] = round(item[k], 4) 

    x_axis_ids = [item["id"] for item in data["axes"]["x-axis"]["ticks"]]
    y_axis_ids = [item["id"] for item in data["axes"]["y-axis"]["ticks"]]

    x_labels = [item for item in data["text"] if item["id"] in x_axis_ids]
    x_labels.sort(key=lambda x: x["polygon"]["x0"])
    x_labels = [item["text"].strip() for item in x_labels]
    x_labels_polygons = [item["polygon"] for item in data["text"] if item["id"] in x_axis_ids]
    x_labels_rects = []
    for polygon in x_labels_polygons:
        xs = [polygon["x0"], polygon["x1"], polygon["x2"], polygon["x3"]]
        ys = [polygon["y0"], polygon["y1"], polygon["y2"], polygon["y3"]]

        x_labels_rects.append([min(xs), min(ys), max(xs), max(ys)])

    y_labels = [item for item in data["text"] if item["id"] in y_axis_ids]
    y_labels.sort(key=lambda x: x["polygon"]["y0"])
    y_labels = [item["text"].strip() for item in y_labels]
    y_labels_polygons = [item["polygon"] for item in data["text"] if item["id"] in y_axis_ids]
    y_labels_rects = []
    for polygon in y_labels_polygons:
        xs = [polygon["x0"], polygon["x1"], polygon["x2"], polygon["x3"]]
        ys = [polygon["y0"], polygon["y1"], polygon["y2"], polygon["y3"]]

        y_labels_rects.append([min(xs), min(ys), max(xs), max(ys)])

    gt = {
        "file_name": os.path.join("images", image_name),
        "ground_truth": {
            "gt_parse": {
                "class": data["chart-type"],
                "value": data["data-series"],
                "x_type": data["axes"]["x-axis"]["values-type"],
                "y_type": data["axes"]["y-axis"]["values-type"],
                "x_labels": x_labels,
                "y_labels": y_labels,
                "x_labels_polygons": x_labels_polygons,
                "y_labels_polygons": y_labels_polygons,
                "x_labels_rects": x_labels_rects,
                "y_labels_rects": y_labels_rects,
            }
        },
        "source": data["source"],
    }
    ground_truths.append(gt)

    # TODO: for multi stages pipeline, we need to save the points of each axis and the points of each data series
    # we need to calculate pixel value of each point in data-series based on axis points
    x_data = []
    for tick in data["axes"]["x-axis"]["ticks"]:
        x_data.append(
            {
                "id": tick["id"],
                "x": tick["tick_pt"]["x"],
                "y": tick["tick_pt"]["y"],
            }
        )

    y_data = []
    for tick in data["axes"]["y-axis"]["ticks"]:
        y_data.append(
            {
                "id": tick["id"],
                "x": tick["tick_pt"]["x"],
                "y": tick["tick_pt"]["y"],
            }
        )
    
    # add label to x_data and y_data
    x_id_to_text = {item["id"]: item["text"] for item in data["text"] if item["id"] in x_axis_ids}
    y_id_to_text = {item["id"]: item["text"] for item in data["text"] if item["id"] in y_axis_ids}

    for item in x_data:
        item["text"] = x_id_to_text[item["id"]]
    
    for item in y_data:
        item["text"] = y_id_to_text[item["id"]]

    # x_data_dict = {item["text"]: item for item in x_data}
    # y_data_dict = {item["text"]: item for item in y_data}
    from collections import defaultdict
    x_data_dict = defaultdict(list)
    for item in x_data:
        x_data_dict[item["text"]].append(item)
    
    y_data_dict = defaultdict(list)
    for item in y_data:
        y_data_dict[item["text"]].append(item)

    # calculate pixel values
    x_type = data["axes"]["x-axis"]["values-type"]
    y_type = data["axes"]["y-axis"]["values-type"]

    skip = False
    for item in data["data-series"]:
        try:
            if x_type == "numerical":
                x_value = float(item["x"])
                x_data.sort(key=lambda x: abs(float(x["text"].replace(",", ".").replace("%", "")) - x_value))
                x1 = x_data[0]
                x2 = x_data[1]
                x1_value = float(x1["text"].replace(",", ".").replace("%", ""))
                x2_value = float(x2["text"].replace(",", ".").replace("%", ""))

                # print("x_value: ", x_value, "x1_value: ", x1_value, "x2_value: ", x2_value, "x1: ", x1, "x2: ", x2)
                # TODO: check this, make it similar to pipeline.ipynb
                if x_value > float(x1["text"]):
                    x_pixel = (x_value - x1_value) / (x2_value - x1_value) * (x2["x"] - x1["x"]) + x1["x"]
                else:
                    x_pixel = x1["x"] - (x1_value - x_value) / (x2_value - x1_value) * (x2["x"] - x1["x"])
            else: # categorical
                x_value = item["x"]
                x_pixel = x_data_dict[x_value].pop(0)["x"]
                # print(x_value)
            
            if y_type == "numerical":
                y_value = float(item["y"])
                y_data.sort(key=lambda x: abs(float(x["text"].replace(",", ".").replace("%", "")) - y_value))
                y1 = y_data[0]
                y2 = y_data[1]
                y1_value = float(y1["text"].replace(",", "").replace("%", ""))
                y2_value = float(y2["text"].replace(",", "").replace("%", ""))

                # print("y_value =", y_value, "y1_value =", y1_value, "y2_value =", y2_value, "y1[y] =", y1["y"], "y2[y] =", y2["y"])
                # TODO: check this, make it similar to pipeline.ipynb
                if y_value > y1_value:
                    y_pixel = (y_value - y1_value) / (y2_value - y1_value) * (y2["y"] - y1["y"]) + y1["y"]
                else:
                    y_pixel = y1["y"] - (y1_value - y_value) / (y2_value - y1_value) * (y2["y"] - y1["y"])
            else: # categorical
                y_value = item["y"]
                y_pixel = y_data_dict[y_value].pop(0)["y"]

            item["x_pixel"] = x_pixel
            item["y_pixel"] = y_pixel
        except:
            skip = True
            break
    # # DEBUG: plot data-series points to image
    # image_path = os.path.join(TRAIN_IMAGE_FOLDER, image_name)
    # image = cv2.imread(image_path)

    # for item in data["data-series"]:
    #     cv2.circle(image, (int(item["x_pixel"]), int(item["y_pixel"])), 5, (0, 0, 255), -1)

    # # cv2.imwrite(os.path.join("debug", image_name), image)
    # plt.imshow(image)
    # plt.show()
    # break


    if not skip:
        keypoints_ground_truths[image_name] = {
            "value": data["data-series"],
            "x": data["axes"]["x-axis"]["ticks"],
            "y": data["axes"]["y-axis"]["ticks"],
            "x_label": x_labels_rects,
            "y_label": y_labels_rects,
            "chart_type": data["chart-type"],
        }

    # prepare polygon ground truth
    polygon_text = ""
    for item in data["text"]:
        x0 = item["polygon"]["x0"]
        y0 = item["polygon"]["y0"]
        x1 = item["polygon"]["x1"]
        y1 = item["polygon"]["y1"]
        x2 = item["polygon"]["x2"]
        y2 = item["polygon"]["y2"]
        x3 = item["polygon"]["x3"]
        y3 = item["polygon"]["y3"]
        polygon_text += f"{x0},{y0},{x1},{y1},{x2},{y2},{x3},{y3},dummy_text\n"

    if data["chart-type"] == "horizontal_bar" and os.path.basename(path).startswith("PMC"): # chartinfo data
        x_axis_ids, y_axis_ids = y_axis_ids, x_axis_ids
    x_labels_poly_text = ""
    for item in data["text"]:
        if item["id"] in x_axis_ids:
            x0 = item["polygon"]["x0"]
            y0 = item["polygon"]["y0"]
            x1 = item["polygon"]["x1"]
            y1 = item["polygon"]["y1"]
            x2 = item["polygon"]["x2"]
            y2 = item["polygon"]["y2"]
            x3 = item["polygon"]["x3"]
            y3 = item["polygon"]["y3"]
            x_labels_poly_text += f"{x0},{y0},{x1},{y1},{x2},{y2},{x3},{y3},dummy_text\n"

    y_labels_poly_text = ""
    for item in data["text"]:
        if item["id"] in y_axis_ids:
            x0 = item["polygon"]["x0"]
            y0 = item["polygon"]["y0"]
            x1 = item["polygon"]["x1"]
            y1 = item["polygon"]["y1"]
            x2 = item["polygon"]["x2"]
            y2 = item["polygon"]["y2"]
            x3 = item["polygon"]["x3"]
            y3 = item["polygon"]["y3"]
            y_labels_poly_text += f"{x0},{y0},{x1},{y1},{x2},{y2},{x3},{y3},dummy_text\n"


    text_detection_ground_truths[image_name] = polygon_text
    x_labels_text_detection_ground_truths[image_name] = x_labels_poly_text
    y_labels_text_detection_ground_truths[image_name] = y_labels_poly_text


In [ ]:
# visualize text_detection_ground_truths
import numpy as np
# shuffle
x_labels_text_detection_ground_truths = {k: x_labels_text_detection_ground_truths[k] for k in np.random.permutation(list(x_labels_text_detection_ground_truths.keys()))}
for image_name, polygon_text in x_labels_text_detection_ground_truths.items():
    image_path = os.path.join(TRAIN_IMAGE_FOLDER, image_name)
    image = cv2.imread(image_path)

    for line in polygon_text.splitlines():
        points = line.split(",dummy_text")[0].split(",")
        points = [int(x) for x in points]
        points = np.array(points).reshape(-1, 2)
        cv2.polylines(image, [points], True, (0, 255, 0), 2)

    plt.imshow(image)
    plt.show()
    break

In [ ]:
# figure(figsize=(8, 6), dpi=80)

# plt.imshow(image)
# plt.show()

In [ ]:
num_item_from_source = {}
for item in ground_truths:
    if item["source"] not in num_item_from_source:
        num_item_from_source[item["source"]] = 0
    num_item_from_source[item["source"]] += 1

print("Number of items from each source: ", num_item_from_source)

In [ ]:
# item that has source = "extracted"
source_image_paths = []
for item in ground_truths:
    if item["source"] == "extracted":
        source_image_paths.append(os.path.join("train", item["file_name"]))

In [ ]:
from PIL import Image

Image.open(source_image_paths[10])

In [ ]:
# read train and validation image paths from train_list.txt and val_list.txt
train_image_paths = []
val_image_paths = []

all_image_paths = []
for item in ground_truths:
    all_image_paths.append(os.path.basename(item["file_name"]))

# with open("train_list.txt", "r") as f:
#     for line in f:
#         train_image_paths.append(line.strip())

with open("val_list.txt", "r") as f:
    for line in f:
        val_image_paths.append(line.strip())


# remaining is train_image_paths
train_image_paths = list(set(all_image_paths) - set(val_image_paths))

print("Number of train images: ", len(train_image_paths))
print("Number of val images: ", len(val_image_paths))


In [ ]:
# split ground truth based on this train and validation image paths
train_ground_truths = []
val_ground_truths = []

set_train_image_paths = set(train_image_paths)
set_val_image_paths = set(val_image_paths)

for item in tqdm(ground_truths):
    image_path = os.path.basename(item["file_name"])
    if image_path in set_train_image_paths:
        train_ground_truths.append(item)
    elif image_path in set_val_image_paths:
        val_ground_truths.append(item)
    else:
        raise Exception("Image path not found in train_list.txt or val_list.txt", image_path)

In [ ]:
# # split ground truths into train and validation, validation images is all images from source = "extracted"
# train_ground_truths = []
# val_ground_truths = []

# for item in ground_truths:
#     if item["source"] != "extracted":
#         train_ground_truths.append(item)

# # split 50:50 extracted source images into train and validation
# remain_ground_truths = []
# for item in ground_truths:
#     if item["source"] == "extracted":
#         remain_ground_truths.append(item)

# # set seed for reproducibility
# import random
# random.seed(42)
# random.shuffle(remain_ground_truths)

# for i in range(len(remain_ground_truths)):
#     if i < len(remain_ground_truths) // 2:
#         train_ground_truths.append(remain_ground_truths[i])
#     else:
#         val_ground_truths.append(remain_ground_truths[i])

# print("Number of train ground truths: ", len(train_ground_truths))
# print("Number of val ground truths: ", len(val_ground_truths))

# # print one example
# print(val_ground_truths[0]["ground_truth"]["gt_parse"])

In [ ]:
train_file_names = set([os.path.basename(item["file_name"]) for item in train_ground_truths])
val_file_names = set([os.path.basename(item["file_name"]) for item in val_ground_truths])

In [ ]:
# split keypoints ground truths into train and validation
train_keypoints_ground_truths = {}
val_keypoints_ground_truths = {}

for image_name, keypoints in keypoints_ground_truths.items():
    if image_name in train_file_names:
        train_keypoints_ground_truths[image_name] = keypoints
    elif image_name in val_file_names:
        val_keypoints_ground_truths[image_name] = keypoints

# # save to json 
# with open(os.path.join(ROOT_DATA_FOLDER, "train_keypoints_ground_truths.json"), "w") as f:
#     json.dump(train_keypoints_ground_truths, f)

# with open(os.path.join(ROOT_DATA_FOLDER, "val_keypoints_ground_truths.json"), "w") as f:
#     json.dump(val_keypoints_ground_truths, f)

In [ ]:
# !pip install jsonlines

In [ ]:
# copy all images to train/images and val/images
import shutil

# make dirs for validation
os.makedirs(os.path.join(ROOT_DATA_FOLDER, "validation/images/"), exist_ok=True)

for gt in tqdm(val_ground_truths):
    shutil.copy2(os.path.join(ROOT_DATA_FOLDER, "train", gt["file_name"]), os.path.join(ROOT_DATA_FOLDER, "validation/images/"))


In [ ]:
# save to jsonl file
import jsonlines

with jsonlines.open(os.path.join(ROOT_DATA_FOLDER, "train/metadata.jsonl"), mode="w") as writer:
    writer.write_all(train_ground_truths)

with jsonlines.open(os.path.join(ROOT_DATA_FOLDER, "validation/metadata.jsonl"), mode="w") as writer:
    writer.write_all(val_ground_truths)

In [ ]:
# temporary
text_gts = y_labels_text_detection_ground_truths

val_image_names = os.listdir(os.path.join(ROOT_DATA_FOLDER, "validation/images"))
train_image_names = set(text_gts.keys()) - set(val_image_names)

print("train_image_names: ", len(train_image_names))
print("val_image_names: ", len(val_image_names))

TRAIN_TEXT_DETECTION_FOLDER = os.path.join(ROOT_DATA_FOLDER, "y_labels_train_gts")
VAL_TEXT_DETECTION_FOLDER = os.path.join(ROOT_DATA_FOLDER, "y_labels_val_gts")

# makedirs
os.makedirs(TRAIN_TEXT_DETECTION_FOLDER, exist_ok=True)
os.makedirs(VAL_TEXT_DETECTION_FOLDER, exist_ok=True)

for image_name, text in tqdm(text_gts.items()):
    if image_name in train_image_names:
        with open(os.path.join(TRAIN_TEXT_DETECTION_FOLDER, image_name + ".txt"), "w") as f:
            f.write(text)
    else:
        with open(os.path.join(VAL_TEXT_DETECTION_FOLDER, image_name + ".txt"), "w") as f:
            f.write(text)

# train_list.txt and val_list.txt
with open(os.path.join(ROOT_DATA_FOLDER, "y_labels_train_list.txt"), "w") as f:
    for image_name in train_image_names:
        f.write(image_name + "\n")

with open(os.path.join(ROOT_DATA_FOLDER, "y_labels_val_list.txt"), "w") as f:
    for image_name in val_image_names:
        f.write(image_name + "\n") 

In [ ]:
# temporary
text_gts = x_labels_text_detection_ground_truths

val_image_names = os.listdir(os.path.join(ROOT_DATA_FOLDER, "validation/images"))
train_image_names = set(text_gts.keys()) - set(val_image_names)

print("train_image_names: ", len(train_image_names))
print("val_image_names: ", len(val_image_names))

TRAIN_TEXT_DETECTION_FOLDER = os.path.join(ROOT_DATA_FOLDER, "x_labels_train_gts")
VAL_TEXT_DETECTION_FOLDER = os.path.join(ROOT_DATA_FOLDER, "x_labels_val_gts")

# makedirs
os.makedirs(TRAIN_TEXT_DETECTION_FOLDER, exist_ok=True)
os.makedirs(VAL_TEXT_DETECTION_FOLDER, exist_ok=True)

for image_name, text in tqdm(text_gts.items()):
    if image_name in train_image_names:
        with open(os.path.join(TRAIN_TEXT_DETECTION_FOLDER, image_name + ".txt"), "w") as f:
            f.write(text)
    else:
        with open(os.path.join(VAL_TEXT_DETECTION_FOLDER, image_name + ".txt"), "w") as f:
            f.write(text)

# train_list.txt and val_list.txt
with open(os.path.join(ROOT_DATA_FOLDER, "x_labels_train_list.txt"), "w") as f:
    for image_name in train_image_names:
        f.write(image_name + "\n")

with open(os.path.join(ROOT_DATA_FOLDER, "x_labels_val_list.txt"), "w") as f:
    for image_name in val_image_names:
        f.write(image_name + "\n") 

In [ ]:
max_len = 0
lens = []

for gt in ground_truths:
    l = len(gt["ground_truth"]["gt_parse"]["value"])
    lens.append(l)
    if l > max_len:
        max_len = l

print("Max number of data-series: ", max_len)

In [ ]:
# draw histogram of length of data-series
import matplotlib.pyplot as plt
plt.hist(lens, bins=10)

In [ ]:
import numpy as np
import Levenshtein as lev
from sklearn.metrics import r2_score


def sigmoid2(x):
    return 2 - 2 / (1 + np.exp(-x))


def rmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean(np.square(y_true - y_pred)))


def nrmse(y_true, y_pred):
    if len(y_true) != len(y_pred):
        return 0
    # y_bar = np.array([np.mean(y_true) for _ in range(len(y_true))])
    # return sigmoid2(rmse(y_true, y_pred) / rmse(y_true, y_bar))
    return sigmoid2(1 - r2_score(y_true, y_pred))


def nlev(y_true, y_pred):
    if len(y_true) != len(y_pred):
        return 0
    return sigmoid2(sum([lev.distance(y_t, y_p) for y_t, y_p in zip(y_true, y_pred)]) / sum([len(y) for y in y_true]))


def calculate_score(pred, gt):
    if pred["class"] != gt["class"]:
        return 0

    if len(pred["value"]) != len(gt["value"]):
        return 0
    
    if len(pred["value"]) == 0 and len(gt["value"]) == 0:
        return 1

    pred_xs = [x["x"] for x in pred["value"]]
    pred_ys = [x["y"] for x in pred["value"]]

    gt_xs = [x["x"] for x in gt["value"]]
    gt_ys = [x["y"] for x in gt["value"]]

    score = 0
    if isinstance(gt_xs[0], str):
        score += nlev(pred_xs, gt_xs)
    else:
        score += nrmse(pred_xs, gt_xs)

    if isinstance(gt_ys[0], str):
        score += nlev(pred_ys, gt_ys)
    else:
        score += nrmse(pred_ys, gt_ys)

    return score / 2

In [ ]:
calculate_score(
    {
        'class': 'scatter',
        'value': [
            {'x': 1949.4201, 'y': 66.683},
            {'x': 1954.6107, 'y': 66.2785},
            {'x': 1959.9936, 'y': 65.6718},
            {'x': 1964.7997, 'y': 64.0537},
        ]
    },
    {
        'class': 'scatter',
        'value': [
            {'x': 1949.4201, 'y': 6.683},
            {'x': 1954.6107, 'y': 66.2785},
            {'x': 1959.9936, 'y': 65.6718},
            {'x': 1964.7997, 'y': 64.0537},
        ]
    },
)

### Prepare YoloX data

In [ ]:
# convert list of x, y pairs to coco format
def convert_keypoints_data_to_coco_format(gts, folder="train", chart_types=[], available_classes=["value", "x", "y"], box_size=12):
    images = []
    annotations = []

    classes = ["value", "x", "y", "x_label", "y_label"]

    i = 0
    for _, (k, v) in tqdm(enumerate(gts.items())):
        if chart_types and v["chart_type"] not in chart_types:
            continue

        path = os.path.join(ROOT_DATA_FOLDER, f"./{folder}/images/", k)
        img = cv2.imread(path)
        images.append({
            "id": i,
            "file_name": k,
            "width": img.shape[1],
            "height": img.shape[0],
        })

        size = box_size
        box_height = box_width = min(int(size / 640 * img.shape[0]), int(size / 640 * img.shape[1]))

        for c in classes:
            if c not in available_classes:
                continue
            if c == "value":
                for j, point in enumerate(v[c]):
                    bbox = [point["x_pixel"] - box_width // 2, point["y_pixel"] - box_height // 2, box_width, box_height]
                    annotations.append({
                        "id": len(annotations),
                        "image_id": i,
                        "category_id": 1,
                        "bbox": bbox,
                        "iscrowd": 0,
                        "area": 100,
                    })
            elif c in ["x", "y"]:
                for j, point in enumerate(v[c]):
                    bbox = [point["tick_pt"]["x"] - box_width // 2, point["tick_pt"]["y"] - box_height // 2, box_width, box_height]
                    annotations.append({
                        "id": len(annotations),
                        "image_id": i,
                        "category_id": 2 if c == "x" else 3,
                        "bbox": bbox,
                        "iscrowd": 0,
                        "area": 100,
                    })

        i += 1

    coco_annotations = {
        "images": images,
        "annotations": annotations,
        "categories": [
            {
                "id": 1,
                "name": "value",
                "supercategory": "value",
            },
            {
                "id": 2,
                "name": "x",
                "supercategory": "x",
            },
            {
                "id": 3,
                "name": "y",
                "supercategory": "y",
            },
        ],
        "type": "instances",
    }

    return coco_annotations

In [ ]:
# # convert to coco format
# train_coco_annotations = convert_keypoints_data_to_coco_format(train_keypoints_ground_truths, box_size=24)
# val_coco_annotations = convert_keypoints_data_to_coco_format(val_keypoints_ground_truths, box_size=24)

# annotation_folder_name = "annotations_all"

# if not os.path.exists(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name)):
#     os.makedirs(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name))

# # save to json
# with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/train_coco_annotations.json"), "w") as writer:
#     json.dump(train_coco_annotations, writer)

# with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/val_coco_annotations.json"), "w") as writer:
#     json.dump(val_coco_annotations, writer)

In [ ]:
train_coco_annotations = convert_keypoints_data_to_coco_format(train_keypoints_ground_truths, chart_types=[], available_classes=["x", "y"], box_size=24)
val_coco_annotations = convert_keypoints_data_to_coco_format(val_keypoints_ground_truths, chart_types=[], available_classes=["x", "y"], box_size=24)

annotation_folder_name = "annotations_xy"

if not os.path.exists(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name)):
    os.makedirs(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name))

# save to json
with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/train_coco_annotations.json"), "w") as writer:
    json.dump(train_coco_annotations, writer)

with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/val_coco_annotations.json"), "w") as writer:
    json.dump(val_coco_annotations, writer)

In [ ]:
for chart_types in [
    ["scatter"],
    ["line"],
    ["horizontal_bar", "vertical_bar", "dot"]
]:

    if chart_types == ["scatter"]:
        annotation_folder_name = "annotations_scatter_value"
        box_size = 12
    elif chart_types == ["line"]:
        annotation_folder_name = "annotations_line_value"
        box_size = 12
    else:
        annotation_folder_name = "annotations_bar_value"
        box_size = 24

    train_coco_annotations = convert_keypoints_data_to_coco_format(train_keypoints_ground_truths, chart_types=chart_types, available_classes=["value"], box_size=box_size)
    val_coco_annotations = convert_keypoints_data_to_coco_format(val_keypoints_ground_truths, chart_types=chart_types, available_classes=["value"], box_size=box_size)

    if not os.path.exists(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name)):
        os.makedirs(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name))

    # save to json
    with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/train_coco_annotations.json"), "w") as writer:
        json.dump(train_coco_annotations, writer)

    with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/val_coco_annotations.json"), "w") as writer:
        json.dump(val_coco_annotations, writer)

In [ ]:
import os
import json
import cv2

ROOT_DATA_FOLDER = "./"
annotation_folder_name = "annotations_xy"


with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/train_coco_annotations.json"), "r") as writer:
    train_coco_annotations = json.load(writer)

with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/val_coco_annotations.json"), "r") as writer:
    val_coco_annotations = json.load(writer)

In [ ]:
# visualize one image with coco annotations to check if everything is correct
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

def visualize_coco_annotations(coco_annotations, image_id):
    coco = COCO()
    coco.dataset = coco_annotations
    coco.createIndex()

    img = coco.loadImgs(image_id)[0]
    I = cv2.imread(os.path.join(ROOT_DATA_FOLDER, "train/images/", img["file_name"]))
    plt.axis("off")
    plt.imshow(I)

    annIds = coco.getAnnIds(imgIds=img["id"])
    anns = coco.loadAnns(annIds)

    for ann in anns:
        bbox = ann["bbox"]
        # different colors for different classes
        if ann["category_id"] == 1:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor="r", facecolor="none")
        elif ann["category_id"] == 2:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor="g", facecolor="none")
        elif ann["category_id"] == 3:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor="b", facecolor="none")
        elif ann["category_id"] == 4:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor="y", facecolor="none")
        elif ann["category_id"] == 5:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor="c", facecolor="none")
        plt.gca().add_patch(rect)
    
    # increase size of image
    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.show()

index = random.randint(0, len(train_coco_annotations["images"]))
visualize_coco_annotations(train_coco_annotations, index)

print(train_coco_annotations["images"][index])

In [ ]:
len(val_coco_annotations["images"])

In [ ]:
len(train_coco_annotations["images"])

In [ ]:
# # wrong data: 374fcc8c9542.jpg, a80688cb2101.jpg, b9507ab4e9fc.jpg, 3d0b1f8e6f61.jpg, 907a4cbc9ff1.jpg, fbf4820dc2c1.jpg

# image_name = "374fcc8c9542.jpg"
# points = train_keypoints_ground_truths[image_name]["value"]

# # visualize 
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# def visualize_points(img, points):
#     plt.axis("off")
#     plt.imshow(img)

#     for point in points:
#         rect = patches.Rectangle((point["x_pixel"] - 5, point["y_pixel"] - 5), 10, 10, linewidth=1, edgecolor="r", facecolor="none")
#         plt.gca().add_patch(rect)
    
#     # increase size of image
#     fig = plt.gcf()
#     fig.set_size_inches(18.5, 10.5)
#     plt.show()

# img = cv2.imread(os.path.join("./train/images/", image_name))
# visualize_points(img, points)


# External data

In [ ]:
# copy all original data to new folder
import shutil

task = "db"

TARGET_IMAGE_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/images"
TARGET_ANNOTATION_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/annotations"

SOURCE_IMAGE_FOLDER = "/home/thanh/bmga/data/train/images"
SOURCE_ANNOTATION_FOLDER = "/home/thanh/bmga/data/train/annotations"

for file in os.listdir(SOURCE_IMAGE_FOLDER):
    shutil.copy(os.path.join(SOURCE_IMAGE_FOLDER, file), TARGET_IMAGE_FOLDER)

for file in os.listdir(SOURCE_ANNOTATION_FOLDER):
    shutil.copy(os.path.join(SOURCE_ANNOTATION_FOLDER, file), TARGET_ANNOTATION_FOLDER)


# !cp -r /home/thanh/bmga/data/validation /home/thanh/bmga/data/external_data/all/validation
!cp -r /home/thanh/bmga/data/validation /home/thanh/bmga/data/external_data/db/validation

In [ ]:
# # !cd external_data/chartinfo && \
# #     unzip -qq ICDAR2023_CHARTINFO_IIITH_UB_UNITEC_PMC_TEST_TASK3_v1.0.zip && \
# #     unzip -qq ICDAR2023_CHARTINFO_IIITH_UB_UNITEC_PMC_TEST_v1.0___NO_TASK3.zip && \
# #     unzip -qq ICDAR2023_CHARTINFO_UB_UNITEC_PMC_TRAIN_V1.0.zip

# !cd external_data/chartqa && unzip -qq ChartQA\ Dataset.zip

# !cd external_data/dvqa && tar -xzf images.tar.gz && tar -xzf metadata.tar.gz

# !cd external_data/figureqa && \
#     tar -xzf figureqa-sample-train-v1.tar.gz && \
#     tar -xzf figureqa-test1-v1.tar.gz && \
#     tar -xzf figureqa-test2-v1.tar.gz && \
#     tar -xzf figureqa-train1-v1.tar.gz && \
#     tar -xzf figureqa-validation1-v1.tar.gz && \
#     tar -xzf figureqa-validation2-v1.tar.gz

# !cd external_data/plotqa/Images/Train && tar -xzf png.tar.gz
# !cd external_data/plotqa/Images/Test && tar -xzf png.tar.gz
# !cd external_data/plotqa/Images/Validation && tar -xzf png.tar.gz

### Graph classification

In [ ]:
pass

### Object detection

In [ ]:
# we should convert the data to the competition format then use the old code to generate the dataset

In [ ]:
import os
import cv2
import json
import numpy as np

from PIL import Image
from tqdm import tqdm


In [ ]:
# Chartinfo
def extract_chartinfo_data(root_annotation_folder, root_image_folder, extracted_annotation_folder, task="all"):
    if not os.path.exists(extracted_annotation_folder):
        os.makedirs(extracted_annotation_folder)

    if task == "all":
        usable_graph_types = ["line", "scatter", "scatter-line", "vertical_bar"] # ignore horizontal_bar because of data issue
    elif task == "db":
        usable_graph_types = ["area", "heatmap", "horizontal_bar", "line", "scatter", "scatter-line", "vertical_bar", "vertical_box", "vertical_interval"]
    mapping = {
        "horizontal_bar": "horizontal_bar",
        "line": "line",
        "scatter": "scatter",
        "scatter-line": "scatter",
        "vertical_bar": "vertical_bar"
    }

    count = 0
    total = 0
    extracted_image_paths = []

    for graph_type in usable_graph_types:
        annotation_folder = os.path.join(root_annotation_folder, graph_type)
        image_folder = os.path.join(root_image_folder, graph_type)

        for annotation_file in tqdm(os.listdir(annotation_folder)):
            total += 1
            data = json.load(open(os.path.join(annotation_folder, annotation_file)))
            # task1 output is graph type
            # task2 output is text location
            # task3 output is type of text
            # task4 output is x/y axis points
            # task5 is legends analysis
            # task6 output is final outputs
            # text data

            if data.get("task4") is None:
                continue

            id_to_role = {
                item["id"]: item["role"] for item in data["task3"]["output"]["text_roles"]
            }
            text_data = []
            for item in data["task2"]["output"]["text_blocks"]:
                item["role"] = id_to_role[item["id"]]
                text_data.append(item)

            # axes data
            axes_data = {
                "x-axis": {
                    "ticks": data["task4"]["output"]["axes"]["x-axis"]
                },
                "y-axis": {
                    "ticks": data["task4"]["output"]["axes"]["y-axis"]
                }
            }
            if graph_type in ["scatter", "scatter-line"]:
                axes_data["x-axis"]["values-type"] = "numerical"
                axes_data["y-axis"]["values-type"] = "numerical"
            elif graph_type in ["vertical_bar", "line"]:
                axes_data["x-axis"]["values-type"] = "categorical"
                axes_data["y-axis"]["values-type"] = "numerical"
            elif graph_type in ["horizontal_bar"]:
                axes_data["x-axis"]["values-type"] = "numerical"
                axes_data["y-axis"]["values-type"] = "categorical"
            else:
                axes_data["x-axis"]["values-type"] = "categorical"
                axes_data["y-axis"]["values-type"] = "categorical"

            if task == "all":
                if data.get("task4") is not None and data.get("task6") is not None and len(data["task6"]["output"]["data series"]) == 1:
                    image_path = os.path.join(image_folder, os.path.splitext(annotation_file)[0] + ".jpg")
                    count += 1

                    data_series = data["task6"]["output"]["data series"][0]["data"]
                else:
                    continue
            elif task == "db":
                image_path = os.path.join(image_folder, os.path.splitext(annotation_file)[0] + ".jpg")
                count += 1

                data_series = []
        
            extracted_annotation = {
                "source": "external",
                "chart-type": mapping.get(graph_type, graph_type),
                "text": text_data,
                "axes": axes_data,
                "data-series": data_series,
            }
            
            # save annotation
            with open(os.path.join(extracted_annotation_folder, annotation_file), "w") as f:
                json.dump(extracted_annotation, f, indent=4)

            extracted_image_paths.append(image_path)
        
    print(f"Extracted {count} images out of {total} images")

    return extracted_image_paths

# # training set: 
root_annotation_folder = "./external_data/chartinfo/annotations_JSON/"
root_image_folder = "./external_data/chartinfo/images/"
extracted_annotation_folder = "./external_data/chartinfo/extracted_annotations_db/"


extracted_image_paths = extract_chartinfo_data(root_annotation_folder, root_image_folder, extracted_annotation_folder, task=task)

if task == "all":
    # write extracted_image_paths to a file
    with open("./external_data/chartinfo/extracted_image_paths.txt", "w") as f:
        for path in extracted_image_paths:
            f.write(path + "\n")
elif task == "db":
    # write extracted_image_paths to a file
    with open("./external_data/chartinfo/extracted_image_paths_db.txt", "w") as f:
        for path in extracted_image_paths:
            f.write(path + "\n")

In [ ]:
import shutil


# copy to all folder
TARGET_IMAGE_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/images"
TARGET_ANNOTATION_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/annotations"

if not os.path.exists(TARGET_IMAGE_FOLDER):
    os.makedirs(TARGET_IMAGE_FOLDER)

if not os.path.exists(TARGET_ANNOTATION_FOLDER):
    os.makedirs(TARGET_ANNOTATION_FOLDER)

for path in tqdm(extracted_image_paths):
    shutil.copy2(path, TARGET_IMAGE_FOLDER)

for file in tqdm(os.listdir(extracted_annotation_folder)):
    shutil.copy2(os.path.join(extracted_annotation_folder, file), TARGET_ANNOTATION_FOLDER)

In [ ]:

# # test set: doesn't have final outputs
# root_test_folder = "./external_data/chartinfo/ICDAR2023_CHARTINFO_IIIT_UB_UNITEC_PMC_TEST_v1.0/"
# test_extracted_image_paths = []

# for folder in os.listdir(root_test_folder):
#     root_annotation_folder = os.path.join(root_test_folder, folder, "annotations")
#     root_image_folder = os.path.join(root_test_folder, folder, "images")

In [ ]:
# FigureQA: generated locally
annotation_folder = "/home/thanh/bmga/data/FigureQA/bmga_generation/figureqa-train1/train1_1/figure_data/json_annotations"
image_folder = "/home/thanh/bmga/data/FigureQA/bmga_generation/figureqa-train1/train1_1/figure_data/png"
extracted_annotation_folder = "/home/thanh/bmga/data/FigureQA/bmga_final/figureqa-train1/extracted_annotations/"

if not os.path.exists(extracted_annotation_folder):
    os.makedirs(extracted_annotation_folder)

mapping = {
    "dot_line": "scatter",
    "hbar_categorical": "horizontal_bar",
    "line": "line",
    "pie": "pie",
    "vbar_categorical": "vertical_bar"
}

# find all annotation - image pairs
annotation_path_dict = {}
for annotation_file in os.listdir(annotation_folder):
    annotation_path_dict[annotation_file.split("_annotations.")[0]] = os.path.join(annotation_folder, annotation_file)

image_path_dict = {}
for image_file in os.listdir(image_folder):
    image_path_dict[image_file.split(".")[0]] = os.path.join(image_folder, image_file)

intersection = set(annotation_path_dict.keys()).intersection(set(image_path_dict.keys()))
print(f"Found {len(intersection)} annotation - image pairs")

image_annotation_pairs = []
for key in intersection:
    image_annotation_pairs.append((image_path_dict[key], annotation_path_dict[key]))

# with open(annotation_path, "r") as f:
#     data = json.load(f)

# TODO: map the points correspond to the labels in the line graph instead of all the points
extracted_image_paths = []
# for item in tqdm(data):
for image_path, annotation_path in tqdm(image_annotation_pairs):
    with open(annotation_path, "r") as f:
        item = json.load(f)
    text_data = []

    graph_type = mapping[item["type"]]
    
    x_id_to_value = {}
    y_id_to_value = {}
    x_id_to_pixel = {}
    y_id_to_pixel = {}

    x_major_label = item["general_figure_info"]["x_axis"]["major_labels"]
    num_x_labels = len(x_major_label["bboxes"]) // 2
    for i in range(num_x_labels):
        box, value = x_major_label["bboxes"][i], x_major_label["values"][i]
        x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])

        text_data.append({
            "id": i,
            "polygon": {
                "x0": x,
                "y0": y,
                "x1": x + w,
                "y1": y,
                "x2": x + w,
                "y2": y + h,
                "x3": x,
                "y3": y + h
            },
            "text": value,
            "role": "tick_label"
        })
        x_id_to_value[i] = value

    y_major_label = item["general_figure_info"]["y_axis"]["major_labels"]
    num_y_labels = len(y_major_label["bboxes"]) // 2
    for i in range(num_y_labels):
        box, value = y_major_label["bboxes"][i], y_major_label["values"][i]
        x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])

        text_data.append({
            "id": num_x_labels + i,
            "polygon": {
                "x0": x,
                "y0": y,
                "x1": x + w,
                "y1": y,
                "x2": x + w,
                "y2": y + h,
                "x3": x,
                "y3": y + h
            },
            "text": value,
            "role": "tick_label"
        })
        y_id_to_value[num_x_labels + i] = value

    axes_data = {}

    x_ticks = item["general_figure_info"]["x_axis"]["major_ticks"]
    num_x_ticks = len(x_ticks["bboxes"]) // 2

    x_axes_data = []
    for i in range(num_x_ticks):
        box = x_ticks["bboxes"][i]
        x, y, w, h = box["x"], box["y"], box["w"], box["h"]

        x_axes_data.append({
            "id": i,
            "tick_pt": {
                "x": x + w // 2,
                "y": int(y + h // 4)
            }
        })
        x_id_to_pixel[i] = x + w // 2

    axes_data["x-axis"] = {
        "ticks": x_axes_data,
        "values-type": "categorical" if graph_type in ["vertical_bar", "line"] else "numerical"  # horizontal_bar, scatter
    }

    y_ticks = item["general_figure_info"]["y_axis"]["major_ticks"]
    num_y_ticks = len(y_ticks["bboxes"]) // 2

    y_axes_data = []
    for i in range(num_y_ticks):
        box = y_ticks["bboxes"][i]
        x, y, w, h = box["x"], box["y"], box["w"], box["h"]

        y_axes_data.append({
            "id": num_x_ticks + i,
            "tick_pt": {
                "x": int(x + 3 * w // 4),
                "y": y + h // 2
            }
        })
        y_id_to_pixel[num_x_ticks + i] = y + h // 2

    axes_data["y-axis"] = {
        "ticks": y_axes_data,
        "values-type": "categorical" if graph_type in ["horizontal_bar"] else "numerical"  # vertical_bar, scatter, line
    }

    if graph_type != "line":
        data_series = [
            {
                "x": x,
                "y": y,
            } for x, y in zip(item["models"][0]["x"], item["models"][0]["y"])
        ]
    else:  # TODO: for line data
        x_value_to_pixel = {}
        for x_id, x_value in x_id_to_value.items():
            x_value_to_pixel[x_value] = x_id_to_pixel[x_id]

        y_value_to_pixel = {}
        for y_id, y_value in y_id_to_value.items():
            y_value_to_pixel[y_value] = y_id_to_pixel[y_id]

        # draw the line in data series then select the values on x labels
        xys = [(x, y) for x, y in zip(item["models"][0]["x"], item["models"][0]["y"])]
        xys.sort(key=lambda x: x[0])

        data_series = []
        for x_value in x_id_to_value.values():
            # find the closest 2 points which have to the x_value in xys
            p1, p2 = sorted(xys, key=lambda x: abs(int(x[0]) - int(x_value)))[:2]

            x1, y1 = p1
            x2, y2 = p2

            # calculate the y_value
            y_value = (y2 - y1) / (x2 - x1) * (int(x_value) - x1) + y1

            data_series.append({
                "x": x_value,
                "y": y_value,
            })


    extracted_annotation = {
        "source": "external",
        "chart-type": graph_type,
        "text": text_data,
        "axes": axes_data,
        "data-series": data_series,
    }

    # save annotation
    # annotation_file = str(item["image_index"]) + ".json"
    annotation_file = os.path.basename(annotation_path).split("_annotations.")[0] + ".json"
    with open(os.path.join(extracted_annotation_folder, annotation_file), "w") as f:
        json.dump(extracted_annotation, f, indent=4)

    # image_path = os.path.join(image_folder, str(item["image_index"]) + ".png")
    extracted_image_paths.append(image_path)

    # if graph_type == "line":
    #     break


# write extracted_image_paths to a file
with open("/home/thanh/bmga/data/FigureQA/bmga_generation/figureqa-train1/train1_1/figure_data/extracted_image_paths.txt", "w") as f:
    for path in extracted_image_paths:
        f.write(path + "\n")

    

In [ ]:
import shutil


# copy to all folder
TARGET_IMAGE_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/images"
TARGET_ANNOTATION_FOLDER = f"/home/thanh/bmga/data/external_data/{task}/train/annotations"

for path in tqdm(extracted_image_paths):
    shutil.copy2(path, TARGET_IMAGE_FOLDER)

for file in tqdm(os.listdir(extracted_annotation_folder)):
    shutil.copy2(os.path.join(extracted_annotation_folder, file), TARGET_ANNOTATION_FOLDER)

In [ ]:

# image = cv2.imread(image_path)

# # visualize data_series
# for data_point in data_series:
#     # print(x, y)
#     x, y = data_point["x"], data_point["y"]

# from matplotlib import pyplot as plt
# # figure size 
# plt.figure(figsize=(10, 10))
# plt.imshow(image)
# data_series

In [ ]:
# # chartqa # doesn't have text for y_labels and ticks information, so we can only use x_labels, final outputs -> maybe for test set and Donut
# root_annotation_folder = "./external_data/chartqa/ChartQA Dataset/train/annotations"
# root_image_folder = "./external_data/chartqa/ChartQA Dataset/train/png"
# extracted_annotation_folder = "./external_data/chartqa/ChartQA Dataset/extracted_annotations/"

# if not os.path.exists(extracted_annotation_folder):
#     os.makedirs(extracted_annotation_folder)

# mapping = {
#     "hbar": "horizontal_bar",
#     "line": "line",
#     "vbar": "vertical_bar",
# }

# count = 0
# total = 0
# graph_types = set()
# for annotation_file in tqdm(os.listdir(root_annotation_folder)):
#     with open(os.path.join(root_annotation_folder, annotation_file)) as f:
#         data = json.load(f)

#     graph_types.add(data["type"])

#     total += 1

#     if len(data["models"]) == 1 and data["type"] in mapping:
#         count += 1
#         image_path = os.path.join(root_image_folder, annotation_file.split(".")[0] + ".png")
#         extracted_annotation = {
#             "source": "external",
#         }

#         break

# data


In [ ]:
# # dvqa: only have boxes for x_labels, y_labels, final info -> can only be used for DB model, Donut
# train_annotation_path = "./external_data/dvqa/metadata/train_metadata.json"
# val_easy_annotation_path = "./external_data/dvqa/metadata/val_easy_metadata.json"
# val_hard_annotation_path = "./external_data/dvqa/metadata/val_hard_metadata.json"

# train_image_folder = "./external_data/dvqa/images/"

# extracted_annotation_folder = "./external_data/dvqa/extracted_annotations/"

# if not os.path.exists(extracted_annotation_folder):
#     os.makedirs(extracted_annotation_folder)

# mapping = {}

# count = 0
# total = 0
# extracted_image_paths = []

# with open(train_annotation_path, "r") as f:
#     train_data = json.load(f)
# # visualize one sample image
# # all_image_paths = [os.path.join(train_image_folder, image_name) for image_name in os.listdir(train_image_folder) ]
# image_path = all_image_paths[1123]
# Image.open(image_path)

In [ ]:
# # figureqa # all synthetic from here: https://github.com/Maluuba/FigureQA
# annotation_path = "/home/thanh/bmga/data/external_data/figureqa/train1/annotations.json"
# image_folder = "/home/thanh/bmga/data/external_data/figureqa/train1/png/"
# extracted_annotation_folder = "/home/thanh/bmga/data/external_data/figureqa/train1/extracted_annotations/"

# if not os.path.exists(extracted_annotation_folder):
#     os.makedirs(extracted_annotation_folder)

# # {'dot_line', 'hbar_categorical', 'line', 'pie', 'vbar_categorical'}
# # mapping = {
# #     "dot_line": "line",


# with open(annotation_path, "r") as f:
#     data = json.load(f)

# image_path = os.path.join(image_folder, str(data[0]["image_index"]) + ".png")
# Image.open(image_path)

In [ ]:
# # plotqa: seems like all synthetic
# annotation_path = "./external_data/plotqa/Annotations/Train/annotations.json"
# image_folder = "./external_data/plotqa/Images/Train/png"

# with open(annotation_path, "r") as f:
#     data = json.load(f)

# image_path = os.path.join(image_folder, str(150000) + ".png")
# Image.open(image_path)

### X/Y labels detection

In [ ]:
# we can take all data from chartinfo without filtering, also data in test set

### Text recognition - parseq

In [ ]:
pass

### New Annotations

In [ ]:
import os
import json

IMAGE_FOLDER = "./train/images"
ANNOTATION_FOLDER = "./train/annotations"

In [ ]:
from matplotlib import pyplot as plt
import cv2

i = 0
# i += 1
annotation_files = os.listdir(ANNOTATION_FOLDER)
while i < len(annotation_files):
    annotation_file = annotation_files[i]
    with open(os.path.join(ANNOTATION_FOLDER, annotation_file), "r") as f:
        data = json.load(f)

    if data["chart-type"] != "scatter" or data["source"] == "generated":
        i += 1
        continue

    points = data["visual-elements"]["scatter points"][0]
    image_name = annotation_file.split(".")[0] + ".jpg"

    # visualize points on image
    image_path = os.path.join(IMAGE_FOLDER, image_name)
    image = cv2.imread(image_path)

    for point in points:
        x, y = int(point["x"]), int(point["y"])
        cv2.circle(image, (x, y), 3, (0, 0, 255), -1)

    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()
    break

In [ ]:
# convert to coco format
annotation_files = os.listdir(ANNOTATION_FOLDER)

# split train val based on train_image_names
train_annotation_files = []
val_annotation_files = []

for file in annotation_files:
    image_name = file.split(".")[0] + ".jpg"
    if image_name in train_file_names:
        train_annotation_files.append(file)
    else:
        val_annotation_files.append(file)

In [ ]:
print(len(train_annotation_files), len(val_annotation_files))

In [ ]:
def convert_to_coco_format(annotation_files):
    i = 0
    image_index = 0

    images = []
    annotations = []

    while i < len(annotation_files):
        annotation_file = annotation_files[i]
        with open(os.path.join(ANNOTATION_FOLDER, annotation_file), "r") as f:
            data = json.load(f)

        if data["chart-type"] != "scatter":
            i += 1
            continue

        points = data["visual-elements"]["scatter points"][0]
        image_name = annotation_file.split(".")[0] + ".jpg"

        # visualize points on image
        image_path = os.path.join(IMAGE_FOLDER, image_name)
        img = cv2.imread(image_path)

        images.append({
            "id": image_index,
            "file_name": image_name,
            "width": img.shape[1],
            "height": img.shape[0],
        })

        size = 12
        box_height = box_width = min(int(size / 640 * img.shape[0]), int(size / 640 * img.shape[1]))

        for point in points:
            x, y = int(point["x"]), int(point["y"])
            bbox = [x - box_width // 2, y - box_height // 2, box_width, box_height]
            annotations.append({
                "id": len(annotations),
                "image_id": image_index,
                "category_id": 1,
                "bbox": bbox,
                "iscrowd": 0,
                "area": 100,
            })

        image_index += 1
        i += 1

    coco_annotations = {
        "images": images,
        "annotations": annotations,
        "categories": [
            {
                "id": 1,
                "name": "value",
                "supercategory": "value",
            },
            {
                "id": 2,
                "name": "x",
                "supercategory": "x",
            },
            {
                "id": 3,
                "name": "y",
                "supercategory": "y",
            },
        ],
        "type": "instances",
    }

    return coco_annotations


train_coco_annotations = convert_to_coco_format(train_annotation_files)
val_coco_annotations = convert_to_coco_format(val_annotation_files)

In [ ]:
annotation_folder_name = "new_scatter_annotations"

if not os.path.exists(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name)):
    os.makedirs(os.path.join(ROOT_DATA_FOLDER, annotation_folder_name))

# save to json
with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/train_coco_annotations.json"), "w") as writer:
    json.dump(train_coco_annotations, writer)

with open(os.path.join(ROOT_DATA_FOLDER, f"{annotation_folder_name}/val_coco_annotations.json"), "w") as writer:
    json.dump(val_coco_annotations, writer)